In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils
from IPython.display import Image
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, Dense, BatchNormalization, Flatten, Conv2D, Dropout
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from sklearn.model_selection import train_test_split 
path = r"D:\Uon_acads\Sem_2\MLiS"

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print(tf.config.experimental.list_physical_devices())


In [ ]:
import os
import pandas as pd
import shutil
import numpy as np

In [ ]:
bs = 128 # batch size
img_width = 224 # input img size
img_height = img_width
val_spilt = 0.2 # fraction of training data used for validation
lr = 1e-3 # learning rate

In [ ]:
# FUNCTION TO APPEND '.png' extension to all the image ids
def append_ext(fn):
    return fn+".png"

In [ ]:
def read_dataset(img_path,label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img,channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img,[img_width,img_height])
    return img,label

def add_noise(img):
    noise = tf.random.normal(tf.shape(img),0,5)
    img += noise
    img = tf.clip_by_value(img, 0, 255)
    return img

In [ ]:
df = pd.read_csv(path +"\\" + r'training_norm_added.csv')
img_path = []
angle = []
speed = []
for _,row in df.iterrows():
    img_path.append(path + "\\" + r'training_data_added\training_data' + "\\" + str(int(row["image_id"])) + r'.png')
    angle.append(row["angle"])
    speed.append(int(row["speed"]))
    
train_im, valid_im, train_lab, valid_lab = train_test_split(img_path, speed, test_size=0.20,
                                                            random_state=40, shuffle = True)

    


In [ ]:
#ds_train = tf.data.Dataset.from_tensor_slices((train_im, {'angle':train_lab[:,0],'speed':train_lab[:,1]}))

#ds_train = tf.data.Dataset.from_tensor_slices((train_im, {'speed':train_lab}))
ds_train = tf.data.Dataset.from_tensor_slices((train_im, {'angle':train_lab}))
ds_train = ds_train.map(read_dataset).cache().map(lambda image, label: (tf.image.random_contrast(image, lower = 0.9, upper = 1.1), label)
                                                                ).map(lambda image, label: (tf.image.random_saturation(image, lower = 0.9, upper = 1.1),label)
                                                                     ).map(lambda image, label: (tf.image.random_brightness(image, 0.7 ,1.3),label)
                                                                          ).map(lambda image, label: (add_noise(image), label)
                                                                               ).shuffle(1000).batch(128).repeat(4)

In [ ]:
#ds_valid = tf.data.Dataset.from_tensor_slices((valid_im, {'angle':valid_lab[:,0],'speed':valid_lab[:,1]}))

#ds_valid = tf.data.Dataset.from_tensor_slices((valid_im, {'speed':valid_lab}))
ds_valid = tf.data.Dataset.from_tensor_slices((valid_im, {'angle':valid_lab}))
ds_valid = ds_valid.map(read_dataset).batch(128)
#.cache().map(lambda image, label: (tf.image.random_contrast(image, lower = 0.9, upper = 1), label)
#                                                                ).map(lambda image, label: (tf.image.random_saturation(image, lower = 0.8, upper = 1.2),label)
#                                                                     ).map(lambda image, label: (tf.image.random_brightness(image, 0.5 ,1.5),label)
#                                                                          ).map(lambda image, label: (add_noise(image), label)
#                                                                               ).shuffle(1000).batch(128) #.repeat(1)

In [ ]:
losses = {
 "angle": "mse"
} #,"speed": "binary_crossentropy"

lossWeights = {"angle": 1.0} #, "speed": 1.0

metrics = {"angle": "mean_squared_error"} #,"speed":"accuracy"

In [ ]:
#BUILDING AN EXTRA FEW DIFFERENT LAYERS JOINING THE LAST 10TH LAYER
mirrored_strategy = tf.distribute.MirroredStrategy (devices=["/gpu:0","/cpu:0"])
with mirrored_strategy.scope():
    mobile = tf.keras.applications.MobileNetV3Large(input_shape=[img_width,img_height,3])    
    x = mobile.layers[-10].output

    x = Dropout(0.2)(x)

    x = Conv2D(2,(1,1),strides = (1, 1),kernel_initializer = glorot_uniform(seed=0))(x)

    x = Flatten()(x)

    angle1 = Dense(64, activation='relu', name='angle_hidden')(x)
    #speed1 = Dense(64, activation='relu', name='speed_hidden')(x)

    angle_pred = Dense(1, activation = 'relu', name='angle')(angle1)
    #speed_pred = Dense(1, activation='sigmoid', name='speed')(speed1)

    model = Model(inputs = mobile.input, outputs = [angle_pred]) #, speed_pred

    for layer in model.layers[:-44]: #14,24,37,54
      layer.trainable = False # fALSE
    
    #COMPILE THE NEW MODEL
    model.compile(optimizer="adam", loss=losses, loss_weights=lossWeights, metrics = metrics)
    

In [ ]:
model.summary()

In [ ]:
#COMPILE THE NEW MODEL
#model.compile(optimizer="adam", loss=losses, loss_weights=lossWeights, metrics = metrics)
#model.compile(optimizer=Adam(learning_rate=lr), loss=[MeanSquaredError(), BinaryCrossentropy(from_logits=False)], metrics = [tf.keras.metrics.MeanSquaredError(), 'accuracy'])

In [ ]:
checkpoint_name = "training_angle_only/cp.ckpt"
checkpoint_path = os.path.join(r"D:\Uon_acads\Sem_2\MLiS\models\multi_model_angle",checkpoint_name)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:

model.fit(ds_train, epochs=10, validation_data=ds_valid, callbacks=[cp_callback])

In [ ]:
model.save(r"D:\Uon_acads\Sem_2\MLiS\models\multi_model_angle")
model.save(r"D:\Uon_acads\Sem_2\MLiS\models\multi_model_angle\my_model_angle.h5") 

In [ ]:
new_model = tf.keras.models.load_model(r"D:\Uon_acads\Sem_2\MLiS\models\multi_model_angle\my_model_angle.h5")

In [ ]:
predict = new_model.predict_generator(test_generator)

In [ ]:
predict

In [ ]:
def read_dataset_test(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img,channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img,[224,224])
    img = tf.reshape(img,[1,224,224,3])
    return img

In [ ]:
# TO CREATE PREDICTIONS OF THE TEST DATA
predictions = []
angle_pred = []
speed_pred = []
for i in range(1,1021):
    angle_pred.append(new_model.predict(read_dataset_test(path + "\\" + r'test_data\test_data' + "\\" + str(i) + r'.png')))
    speed_pred.append(model.predict(read_dataset_test(path + "\\" + r'test_data\test_data' + "\\" + str(i) + r'.png')))
    


In [ ]:
for i in range(len(angle_pred)):
    speed = speed_pred[i]
    if abs(1-speed) < abs(speed-0):
        speed = 1
    else:
        speed = 0
    predictions.append([round(angle_pred[i][0][0],4),speed])

In [ ]:
predictions = np.stack(predictions)

In [ ]:
df1 = pd.DataFrame(predictions,index = np.arange(len(predictions))+1, columns=['angle', 'speed'])

In [ ]:
df1.to_csv(path +"\\" + 'multi_model_output.csv')